In [ ]:
!pip install -q transformers datasets bitsandbytes unsloth trl torch==2.3.0
!pip install --upgrade unsloth transformers bitsandbytes datasets trl

In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive')
access_token = userdata.get('HF_TOKEN')

In [ ]:
import unsloth
from unsloth import FastLanguageModel
import torch

max_seq_length = 128
model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype=None,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
    token=access_token,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # LoRA 랭크, 8~16 주로 사용
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "down_proj", "up_proj"
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

from datasets import load_dataset, concatenate_datasets

def load_and_prepare_datasets():
    ds1 = load_dataset("json", data_files={"train": "/content/drive/MyDrive/data/health_train.jsonl"})["train"]
    ds2 = load_dataset("json", data_files={"train": "/content/drive/MyDrive/data/rehab_train.jsonl"})["train"]
    ds3 = load_dataset("json", data_files={"train": "/content/drive/MyDrive/data/security_train.jsonl"})["train"]
    combined = concatenate_datasets([ds1, ds2, ds3])
    return combined

dataset = load_and_prepare_datasets()

def preprocess(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=max_seq_length
      )


dataset = dataset.map(preprocess, batched=True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    args=TrainingArguments(
        output_dir="/content/drive/MyDrive/output_model",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        save_strategy="epoch",
        logging_steps=10,
        report_to=[],
    ),
)

In [ ]:
trainer.train()

In [ ]:
!zip -r /content/drive/MyDrive/output_model.zip /content/drive/MyDrive/output_model/